In [1]:
import numpy as np
from scipy.special import expit
from scipy.special import logsumexp
seed = 0xDEADF00D
np.random.seed(seed)

Загрузим данные:

In [2]:
!wget -O L.npy https://www.dropbox.com/scl/fi/wx98ttwxuelr1fxvvzx0x/L.npy?rlkey=xowj246djriekfc76kie2vk6q&st=97d88zth&dl=0

--2025-08-17 20:02:11--  https://www.dropbox.com/scl/fi/wx98ttwxuelr1fxvvzx0x/L.npy?rlkey=xowj246djriekfc76kie2vk6q
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1001c52e96fffe2aad9ba0fce7.dl.dropboxusercontent.com/cd/0/inline/Cvn4lpwPpXKuCvcYokyoGaHlEVJB6P6Vd3yAr3qu0cJ0bB9utmnzuGGzeUd7YzedAPm29h9JyKAEdF82i20hPu4qcXoo_IzvJgW8dz1_eDL6QQ5TI-F5a1XG5HKkzFpjJdmfBmtiDiUq_IxUd0SDD8US/file# [following]
--2025-08-17 20:02:12--  https://uc1001c52e96fffe2aad9ba0fce7.dl.dropboxusercontent.com/cd/0/inline/Cvn4lpwPpXKuCvcYokyoGaHlEVJB6P6Vd3yAr3qu0cJ0bB9utmnzuGGzeUd7YzedAPm29h9JyKAEdF82i20hPu4qcXoo_IzvJgW8dz1_eDL6QQ5TI-F5a1XG5HKkzFpjJdmfBmtiDiUq_IxUd0SDD8US/file
Resolving uc1001c52e96fffe2aad9ba0fce7.dl.dropboxusercontent.com (uc1001c52e96fffe2aad9ba0fce7.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:60

реализация EM-алгоритма:

In [3]:
L = np.load('L.npy')
n, m = L.shape

def softplus(x):
    '''stable version of log(1 + exp(x))'''
    c = (x > 20) * 1.
    return np.log1p(np.exp(x * (1-c)) * (1-c)) + x * c

def posterior(alpha, beta, L):
    """ Posterior over true labels z p(z|l, \alpha, \beta)
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
    """
    log_prob_z = np.log(0.5)
    log_prob_z0_true = log_prob_z + np.sum((L == 0) * (-softplus(-alpha * beta[:, None])) + (L == 1) * (-softplus(alpha * beta[:, None])), axis=1)
    log_prob_z1_true = log_prob_z + np.sum((L == 1) * (-softplus(-alpha * beta[:, None])) + (L == 0) * (-softplus(alpha * beta[:, None])), axis=1)
    log_prob_z_l = np.stack([log_prob_z0_true, log_prob_z1_true], axis=0)
    log_q = log_prob_z_l - logsumexp(log_prob_z_l, axis=0)
    return np.exp(log_q)


def alpha_grad_lb(alpha, beta, L, q):
    """ Gradient of lower bound wrt alpha
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
        q: ndarray of shape (2, n_problems).
    """
    sigma_alpha_beta = expit(alpha[None, :] * beta[:, None])
    sigma_neg_alfa_beta = expit(-alpha[None, :] * beta[:, None])
    one_class = q[0, :, None] * beta[:, None] * ((L == 0) * sigma_neg_alfa_beta - (L == 1) * sigma_alpha_beta)
    another_class = q[1, :, None] * beta[:, None] * ((L == 1) * sigma_neg_alfa_beta - (L == 0) * sigma_alpha_beta)
    grad = np.sum(one_class + another_class, axis=0)
    return grad


def logbeta_grad_lb(alpha, beta, L, q):
    """ Gradient of lower bound wrt alpha
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
        q: ndarray of shape (2, n_problems).
    """
    sigma_alpha_beta = expit(alpha[None, :] * beta[:, None])
    sigma_neg_alfa_beta = expit(-alpha[None, :] * beta[:, None])
    one_class = q[0, :, None] * alpha[None, :] * ((L == 0) * sigma_neg_alfa_beta - (L == 1) * sigma_alpha_beta)
    another_class = q[1, :, None] * alpha[None, :] * ((L == 1) * sigma_neg_alfa_beta - (L == 0) * sigma_alpha_beta)
    grad = np.sum(one_class + another_class , axis=1)
    return grad


def lower_bound(alpha, beta, L, q):
    """ Lower bound
    Args:
        alpha: ndarray of shape (n_experts).
        beta: ndarray of shape (n_problems).
        L: ndarray of shape (n_problems, n_experts).
        q: ndarray of shape (2, n_problems).
    """
    one_class = q[0, :, None] * ((L == 0) * -softplus(-alpha * beta[:, None]) + (L == 1) * -softplus(alpha * beta[:, None]))
    another_class = q[1, :, None] * ((L == 1) * -softplus(-alpha * beta[:, None]) + (L == 0) * -softplus(alpha * beta[:, None]))
    return np.sum(one_class + another_class) - np.sum(q * np.log(q))

In [4]:
def em(L, n_steps=1000, lr=1e-3):
    # initialize parameters
    alpha, logbeta = np.random.randn(m), np.random.randn(n)

    for step in range(n_steps):
      beta = np.exp(logbeta)
      # E-step
      q = posterior(alpha, beta, L)
      # M-step
      alpha += lr * alpha_grad_lb(alpha, beta, L, q)
      logbeta += lr * logbeta_grad_lb(alpha, logbeta, L, q)
    return alpha, np.exp(logbeta), q

In [5]:
alpha, beta, q = em(L)

In [7]:
!wget -O y.npy https://www.dropbox.com/scl/fi/6k1gj7qslf2n7ziiehjt3/y.npy?rlkey=h8eqstuyobanpkqgb5zcsj1oj&st=m4bs4thw&dl=0

--2025-08-17 20:05:46--  https://www.dropbox.com/scl/fi/6k1gj7qslf2n7ziiehjt3/y.npy?rlkey=h8eqstuyobanpkqgb5zcsj1oj
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc549c3667d3d3b2a92c4f0d5cee.dl.dropboxusercontent.com/cd/0/inline/CvlkuGeoEto9zyLC8WDxiArljC4f_a0b_NcDBAFi1ELGv2dwIiiaUfuXPRdBBtahCWGEKmVJDtkVSrABz7IM4bzqcTTt1u--m3jfY6sfw27UsDXQqUPlXssP4lZzrwGK6N55vin6Hy1prpV3RQrDBZwv/file# [following]
--2025-08-17 20:05:47--  https://uc549c3667d3d3b2a92c4f0d5cee.dl.dropboxusercontent.com/cd/0/inline/CvlkuGeoEto9zyLC8WDxiArljC4f_a0b_NcDBAFi1ELGv2dwIiiaUfuXPRdBBtahCWGEKmVJDtkVSrABz7IM4bzqcTTt1u--m3jfY6sfw27UsDXQqUPlXssP4lZzrwGK6N55vin6Hy1prpV3RQrDBZwv/file
Resolving uc549c3667d3d3b2a92c4f0d5cee.dl.dropboxusercontent.com (uc549c3667d3d3b2a92c4f0d5cee.dl.dropboxusercontent.com)... 162.125.3.15, 2620:100:6026

Рассчитаем `accuracy` разметки, полученной с помощью обычного голосования по большинству среди экспертов, и сравним его с качеством, полученным с помощью EM-алгоритма:

In [8]:
from sklearn.metrics import accuracy_score

y = np.load('y.npy')
accuracy_majority = accuracy_score(y, (np.mean(L, axis=1) >= 0.5).astype(int))
print(f"Accuracy majority: {accuracy_majority }")
accuracy_em = accuracy_score(y, np.argmax(q, axis=0))
if accuracy_em < 0.5:
  alpha = -alpha
  accuracy_em = accuracy_score(y, 1 - np.argmax(q, axis=0))
print(f"Accuracy EM-algorithm: {accuracy_em}")

Accuracy majority: 0.892
Accuracy EM-algorithm: 0.935
